---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [21]:
import pandas as pd
import numpy as np

def blight_model():

    #Read CSV 
    traindata = pd.read_csv('train.csv',encoding='cp1252')
    testdata = pd.read_csv('test.csv',encoding='cp1252')
    address = pd.read_csv('addresses.csv',encoding='cp1252')
    location = pd.read_csv('latlons.csv',encoding='cp1252')
    #Merging Data Frame
    add_loc = address.merge(location, left_on='address', right_on='address')
    traindata = traindata.merge(add_loc[['ticket_id','lat','lon']], left_on='ticket_id', right_on='ticket_id')
    testdata = testdata.merge(add_loc[['ticket_id','lat','lon']], left_on='ticket_id', right_on='ticket_id')
    #Removing "NaT" and Calculating diff 
    traindata['hearing_date'] = traindata['hearing_date'].replace(np.nan,0)
    traindata['ticket_issued_date'] = pd.to_datetime(traindata['ticket_issued_date'])
    traindata['hearing_date'] = pd.to_datetime(traindata['hearing_date'])
    traindata['Diff_Days'] = traindata['hearing_date'] - traindata['ticket_issued_date']
    #Extract Number of Days
    traindata['Diff_Days'] = traindata['Diff_Days'].apply(lambda x: float(x.days))
    
    #Sorting Dataset
    traindata = traindata.sort_values('ticket_id')
    testdata = testdata.sort_values('ticket_id')

    #Reseting and dropping index
    traindata = traindata.reset_index()
    testdata = testdata.reset_index()

    traindata = traindata.drop('index',axis=1)
    testdata = testdata.drop('index',axis=1)
    
    #Dropping Null Values from Compliance 
    
    traindata = traindata[traindata['compliance'].isnull()==False]
    
    #Selecting Dataframe for ML Model  
    df = traindata[['ticket_id','Diff_Days','lat','lon','disposition','fine_amount','discount_amount','judgment_amount','compliance']]
    #Dropping Null Values    
    df = df.dropna()
    disposition_df = pd.get_dummies(df['disposition'])
    
    #Merge DF
    df = df.join(disposition_df)
    #Drop Disposition
    df = df.drop('disposition', axis=1)
    
    #Selecting Columns for ML model
    
    X = df[['Diff_Days','lat', 'lon', 'fine_amount',
       'discount_amount', 'judgment_amount', 'Responsible (Fine Waived) by Deter', 'Responsible by Admission',
       'Responsible by Default', 'Responsible by Determination']]
    y = df['compliance']
    

    from sklearn.model_selection import train_test_split

    from sklearn.ensemble import RandomForestClassifier

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.40)

    rfc = RandomForestClassifier(n_estimators=150).fit(X_train, y_train)
    #Gradien Boosting Model Classifier
#     from sklearn.ensemble import GradientBoostingClassifier
#     from sklearn.model_selection import GridSearchCV
#     from sklearn.metrics import auc
#     X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.40)
#     parameters = {'learning_rate':[0.01,0.1,1,10], 'max_depth':[2,4,6,8,10]}
#     gbc = GradientBoostingClassifier()
#     gbClf = GridSearchCV(gbc, parameters, scoring='roc_auc')
#     gbClf.fit(X_train, y_train)
    
    #Testing Model on test.csv
    
    testdata['hearing_date'] = testdata['hearing_date'].replace(np.nan,0)
    testdata['ticket_issued_date'] = pd.to_datetime(testdata['ticket_issued_date'])
    testdata['hearing_date'] = pd.to_datetime(testdata['hearing_date'])
    testdata['Diff_Days'] = testdata['hearing_date'] - testdata['ticket_issued_date']
    
    testdata[['lat','lon']] = testdata[['lat','lon']].replace(np.nan,0)
    testdata['Diff_Days'] = testdata['Diff_Days'].apply(lambda x: float(x.days))
       #Mapping Values
    
    dispostion_mapping = {'Responsible by Default':'Responsible by Default','Responsible (Fine Waived) by Deter':'Responsible (Fine Waived) by Deter','Responsible by Admission':'Responsible by Admission','Responsible by Determination':'Responsible by Determination','Responsible (Fine Waived) by Admis':'Responsible (Fine Waived) by Deter','Responsible - Compl/Adj by Default':'Responsible by Default', 'Responsible - Compl/Adj by Determi':'Responsible by Determination','Responsible by Dismissal':'Responsible by Determination'}
    testdata['disposition'] = testdata['disposition'].map(dispostion_mapping)
    
    disposition_tdf = pd.get_dummies(testdata['disposition'])
    testdata = testdata.join(disposition_tdf)
    testdf = testdata[['Diff_Days','lat', 'lon', 'fine_amount','discount_amount', 'judgment_amount', 'Responsible (Fine Waived) by Deter', 'Responsible by Admission','Responsible by Default', 'Responsible by Determination']]
    
    #y_pred = gbClf.predict(testdf)
    
    ans = pd.Series(rfc.predict_proba(testdf)[:, 1])
    ans.index = testdata['ticket_id']
    
    return ans
    
    
    

In [82]:
# add_loc[(add_loc['lat'].isnull()) & (add_loc['lon'].isnull())]

,ticket_id,address,lat,lon
142084,89535,"20424 bramford, Detroit MI",NaN,NaN
142085,317124,"20424 bramford, Detroit MI",NaN,NaN
244869,223598,"445 fordyce, Detroit MI",NaN,NaN
269263,280256,"8300 fordyce, Detroit MI",NaN,NaN
287412,329689,"8325 joy rd, Detroit MI 482O4",NaN,NaN
287490,329393,"1201 elijah mccoy dr, Detroit MI 48208",NaN,NaN
290780,333990,"12038 prairie, Detroit MI 482O4",NaN,NaN
308144,367165,"6200 16th st, Detroit MI 482O8",NaN,NaN


In [86]:
# traindata[['Diff_Days','compliance']].where(traindata['Diff_Days'] < 0).dropna()

,Diff_Days,compliance
3096,-12829.0,1.0
3707,-12832.0,1.0
5703,-12846.0,0.0
6321,-12852.0,1.0
6938,-12857.0,0.0
6941,-12858.0,1.0
6978,-12858.0,1.0
8006,-12865.0,0.0
8698,-12872.0,1.0
9686,-12879.0,1.0


In [93]:
# traindata.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance', 'lat', 'lon',
       'Diff_Days'],
      dtype='object')

In [105]:
# df.corr()

,ticket_id,Diff_Days,lat,lon,fine_amount,discount_amount,judgment_amount,compliance,Responsible (Fine Waived) by Deter,Responsible by Admission,Responsible by Default,Responsible by Determination
ticket_id,1.000000,-0.011491,0.020308,0.031988,-0.001441,-0.023590,-0.000766,-0.026264,0.051471,-0.092884,0.132918,-0.099263
Diff_Days,-0.011491,1.000000,0.011624,0.002674,-0.010273,-0.165740,-0.009110,-0.095082,-0.024237,-0.016781,0.049414,-0.053093
lat,0.020308,0.011624,1.000000,0.175336,-0.002891,0.000261,-0.002592,-0.021569,0.006264,-0.027388,0.024222,-0.003856
lon,0.031988,0.002674,0.175336,1.000000,-0.024070,-0.006432,-0.023963,-0.000431,-0.007363,-0.019367,0.017543,-0.001460
fine_amount,-0.001441,-0.010273,-0.002891,-0.024070,1.000000,0.039495,0.999890,-0.049135,-0.018466,-0.064040,0.038036,0.026169
discount_amount,-0.023590,-0.165740,0.000261,-0.006432,0.039495,1.000000,0.033869,0.156073,-0.001596,0.117910,-0.097387,0.001415
judgment_amount,-0.000766,-0.009110,-0.002592,-0.023963,0.999890,0.033869,1.000000,-0.052639,-0.019796,-0.071529,0.044884,0.025253
compliance,-0.026264,-0.095082,-0.021569,-0.000431,-0.049135,0.156073,-0.052639,1.000000,0.124956,0.238997,-0.335454,0.202819
Responsible (Fine Waived) by Deter,0.051471,-0.024237,0.006264,-0.007363,-0.018466,-0.001596,-0.019796,0.124956,1.000000,-0.010699,-0.088560,-0.007831
Responsible by Admission,-0.092884,-0.016781,-0.027388,-0.019367,-0.064040,0.117910,-0.071529,0.238997,-0.010699,1.000000,-0.775862,-0.068603


In [22]:
# import numpy as np
# bm = blight_model()
# res = '{:40s}'.format('Object Type:')
# res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
# res += '{:40s}'.format('Data Shape:')
# res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
# res += '{:40s}'.format('Data Values Type:')
# res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
# res += '{:40s}'.format('Data Values Infinity:')
# res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
# res += '{:40s}'.format('Data Values NaN:')
# res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
# res += '{:40s}'.format('Data Values in [0,1] Range:')
# res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
# res += '{:40s}'.format('Data Values not all 0 or 1:')
# res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
# res += '{:40s}'.format('Index Type:')
# res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
# res += '{:40s}'.format('Index Values:')
# if bm.index.shape==(61001,):
#     res +=['Failed: index values should match test.csv\n','Passed\n'
#           ][all(pd.read_csv('readonly/test.csv',usecols=[0],index_col=0
#                            ).sort_index().index.values==bm.sort_index().index.values)]
# else:
#     res+='Failed: bm.index length should be 61001'
# res += '{:40s}'.format('Can run model twice:')
# bm2 = None
# try:
#     bm2 = blight_model()
#     res += 'Passed\n'
# except:
#     res += ['Failed: second run of blight_model() threw an Exception']
# res += '{:40s}'.format('Can run model twice with same results:')
# if not bm2 is None:
#     res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
#         all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
# print(res)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Object Type:                            Passed
Data Shape:                             Passed
Data Values Type:                       Passed
Data Values Infinity:                   Passed
Data Values NaN:                        Passed
Data Values in [0,1] Range:             Passed
Data Values not all 0 or 1:             Passed
Index Type:                             Passed
Index Values:                           Passed
Can run model twice:                    Passed
Can run model twice with same results:  Failed: second run of blight_model() produced different results (this might not be a problem)

